In [34]:
#!wget http://www.calvin.edu/~stob/data/Berkeley.csv

--2017-10-22 23:51:15--  http://www.calvin.edu/~stob/data/Berkeley.csv
Resolving www.calvin.edu (www.calvin.edu)... 64:ff9b::996a:417, 153.106.4.23
Connecting to www.calvin.edu (www.calvin.edu)|64:ff9b::996a:417|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 543 [text/plain]
Saving to: ‘Berkeley.csv.1’

Berkeley.csv.1      100%[===================>]     543  --.-KB/s    in 0s      

2017-10-22 23:51:16 (23.6 MB/s) - ‘Berkeley.csv.1’ saved [543/543]



In [1]:
import pandas as pd
import scipy.stats as stats

In [2]:
df = pd.read_csv('Berkeley.csv')

In [3]:
df = df.groupby(["Gender", 'Admit']).agg('sum').unstack()
df

Freq         
Admit  Admitted Rejected
Gender                  
Female      557     1278
Male       1198     1493

In [4]:
df = pd.read_csv('Berkeley.csv')
df.pivot_table(columns='Gender', index=['Dept', 'Admit'])

Freq     
Gender        Female Male
Dept Admit               
A    Admitted     89  512
     Rejected     19  313
B    Admitted     17  353
     Rejected      8  207
C    Admitted    202  120
     Rejected    391  205
D    Admitted    131  138
     Rejected    244  279
E    Admitted     94   53
     Rejected    299  138
F    Admitted     24   22
     Rejected    317  351

In [5]:
df = df.pivot_table(columns='Admit', index=['Dept', 'Gender'])
df

Freq         
Admit       Admitted Rejected
Dept Gender                  
A    Female       89       19
     Male        512      313
B    Female       17        8
     Male        353      207
C    Female      202      391
     Male        120      205
D    Female      131      244
     Male        138      279
E    Female       94      299
     Male         53      138
F    Female       24      317
     Male         22      351

In [6]:
df['Total'] = df.sum(axis=1)

In [7]:
df

Freq          Total
Admit       Admitted Rejected      
Dept Gender                        
A    Female       89       19   108
     Male        512      313   825
B    Female       17        8    25
     Male        353      207   560
C    Female      202      391   593
     Male        120      205   325
D    Female      131      244   375
     Male        138      279   417
E    Female       94      299   393
     Male         53      138   191
F    Female       24      317   341
     Male         22      351   373

In [8]:
df.columns

MultiIndex(levels=[['Freq', 'Total'], ['Admitted', 'Rejected', '']],
           labels=[[0, 0, 1], [0, 1, 2]],
           names=[None, 'Admit'])

In [9]:
df.iloc[:, 0] = df.iloc[:, 0] / df.iloc[:, 2]
df.iloc[:, 1] = df.iloc[:, 1] / df.iloc[:, 2]

In [10]:
total = df.unstack().iloc[:, [4, 5]]
total

Total     
Admit             
Gender Female Male
Dept              
A         108  825
B          25  560
C         593  325
D         375  417
E         393  191
F         341  373

In [11]:
df.drop('Total', axis=1, inplace=True)

In [12]:
df.unstack().columns

MultiIndex(levels=[['Freq', 'Total'], ['Admitted', 'Rejected', ''], ['Female', 'Male']],
           labels=[[0, 0, 0, 0], [0, 0, 1, 1], [0, 1, 0, 1]],
           names=[None, 'Admit', 'Gender'])

In [13]:
df_pct = df.unstack().stack(level=1).unstack()
df_pct

Freq                              
Gender    Female                Male          
Admit   Admitted  Rejected  Admitted  Rejected
Dept                                          
A       0.824074  0.175926  0.620606  0.379394
B       0.680000  0.320000  0.630357  0.369643
C       0.340641  0.659359  0.369231  0.630769
D       0.349333  0.650667  0.330935  0.669065
E       0.239186  0.760814  0.277487  0.722513
F       0.070381  0.929619  0.058981  0.941019

In [14]:
df_pct_total = pd.concat([df_pct, total], axis=1)
df_pct_total

Freq                                Total     
Gender    Female                Male                      
Admit   Admitted  Rejected  Admitted  Rejected Female Male
Dept                                                      
A       0.824074  0.175926  0.620606  0.379394    108  825
B       0.680000  0.320000  0.630357  0.369643     25  560
C       0.340641  0.659359  0.369231  0.630769    593  325
D       0.349333  0.650667  0.330935  0.669065    375  417
E       0.239186  0.760814  0.277487  0.722513    393  191
F       0.070381  0.929619  0.058981  0.941019    341  373

In [15]:
stats.ttest_ind(df_pct_total.iloc[:, 0].values, df_pct_total.iloc[:, 2].values)

Ttest_indResult(statistic=0.24771862532494385, pvalue=0.80936433412014097)

In [16]:
stats.ttest_ind(df_pct_total.iloc[:, 1].values, df_pct_total.iloc[:, 3].values)

Ttest_indResult(statistic=-0.24771862532494429, pvalue=0.80936433412014064)

## Problem Set 1: Titanic Survivor Data

In [2]:
import numpy
import pandas
import statsmodels.api as sm

def simple_heuristic(file_path):
    '''
    In this exercise, we will perform some rudimentary practices similar to those of
    an actual data scientist.
    
    Part of a data scientist's job is to use her or his intuition and insight to
    write algorithms and heuristics. A data scientist also creates mathematical models 
    to make predictions based on some attributes from the data that they are examining.

    We would like for you to take your knowledge and intuition about the Titanic
    and its passengers' attributes to predict whether or not the passengers survived
    or perished. You can read more about the Titanic and specifics about this dataset at:
    http://en.wikipedia.org/wiki/RMS_Titanic
    http://www.kaggle.com/c/titanic-gettingStarted
        
    In this exercise and the following ones, you are given a list of Titantic passengers
    and their associated information. More information about the data can be seen at the 
    link below:
    http://www.kaggle.com/c/titanic-gettingStarted/data. 

    For this exercise, you need to write a simple heuristic that will use
    the passengers' gender to predict if that person survived the Titanic disaster.
    
    You prediction should be 78% accurate or higher.
        
    Here's a simple heuristic to start off:
       1) If the passenger is female, your heuristic should assume that the
       passenger survived.
       2) If the passenger is male, you heuristic should
       assume that the passenger did not survive.
    
    You can access the gender of a passenger via passenger['Sex'].
    If the passenger is male, passenger['Sex'] will return a string "male".
    If the passenger is female, passenger['Sex'] will return a string "female".

    Write your prediction back into the "predictions" dictionary. The
    key of the dictionary should be the passenger's id (which can be accessed
    via passenger["PassengerId"]) and the associated value should be 1 if the
    passenger survied or 0 otherwise.

    For example, if a passenger is predicted to have survived:
    passenger_id = passenger['PassengerId']
    predictions[passenger_id] = 1

    And if a passenger is predicted to have perished in the disaster:
    passenger_id = passenger['PassengerId']
    predictions[passenger_id] = 0
    
    You can also look at the Titantic data that you will be working with
    at the link below:
    https://s3.amazonaws.com/content.udacity-data.com/courses/ud359/titanic_data.csv
    '''

    predictions = {}
    df = pandas.read_csv(file_path)
    for passenger_index, passenger in df.iterrows():
        passenger_id = passenger['PassengerId']
      
        # Your code here:
        # For example, let's assume that if the passenger
        # is a male, then the passenger survived.
        #     if passenger['Sex'] == 'male':
        #         predictions[passenger_id] = 1
        if passenger['Sex'] == 'male':
            predictions[passenger_id] = 0
        else:
            predictions[passenger_id] = 1
        
    return predictions


In [3]:
url = 'https://s3.amazonaws.com/content.udacity-data.com/courses/ud359/titanic_data.csv'
simple_heuristic(url)

{1: 0,
 2: 1,
 3: 1,
 4: 1,
 5: 0,
 6: 0,
 7: 0,
 8: 0,
 9: 1,
 10: 1,
 11: 1,
 12: 1,
 13: 0,
 14: 0,
 15: 1,
 16: 1,
 17: 0,
 18: 0,
 19: 1,
 20: 1,
 21: 0,
 22: 0,
 23: 1,
 24: 0,
 25: 1,
 26: 1,
 27: 0,
 28: 0,
 29: 1,
 30: 0,
 31: 0,
 32: 1,
 33: 1,
 34: 0,
 35: 0,
 36: 0,
 37: 0,
 38: 0,
 39: 1,
 40: 1,
 41: 1,
 42: 1,
 43: 0,
 44: 1,
 45: 1,
 46: 0,
 47: 0,
 48: 1,
 49: 0,
 50: 1,
 51: 0,
 52: 0,
 53: 1,
 54: 1,
 55: 0,
 56: 0,
 57: 1,
 58: 0,
 59: 1,
 60: 0,
 61: 0,
 62: 1,
 63: 0,
 64: 0,
 65: 0,
 66: 0,
 67: 1,
 68: 0,
 69: 1,
 70: 0,
 71: 0,
 72: 1,
 73: 0,
 74: 0,
 75: 0,
 76: 0,
 77: 0,
 78: 0,
 79: 0,
 80: 1,
 81: 0,
 82: 0,
 83: 1,
 84: 0,
 85: 1,
 86: 1,
 87: 0,
 88: 0,
 89: 1,
 90: 0,
 91: 0,
 92: 0,
 93: 0,
 94: 0,
 95: 0,
 96: 0,
 97: 0,
 98: 0,
 99: 1,
 100: 0,
 101: 1,
 102: 0,
 103: 0,
 104: 0,
 105: 0,
 106: 0,
 107: 1,
 108: 0,
 109: 0,
 110: 1,
 111: 0,
 112: 1,
 113: 0,
 114: 1,
 115: 1,
 116: 0,
 117: 0,
 118: 0,
 119: 0,
 120: 1,
 121: 0,
 122: 0,
 123: 0,
 

In [4]:
import numpy
import pandas
import statsmodels.api as sm

def complex_heuristic(file_path):
    '''
    You are given a list of Titantic passengers and their associated
    information. More information about the data can be seen at the link below:
    http://www.kaggle.com/c/titanic-gettingStarted/data

    For this exercise, you need to write a more sophisticated algorithm
    that will use the passengers' gender and their socioeconomical class and age 
    to predict if they survived the Titanic diaster. 
    
    You prediction should be 79% accurate or higher.
    
    Here's the algorithm, predict the passenger survived if:
    1) If the passenger is female or
    2) if his/her socioeconomic status is high AND if the passenger is under 18
    
    Otherwise, your algorithm should predict that the passenger perished in the disaster.
    
    Or more specifically in terms of coding:
    female or (high status and under 18)
    
    You can access the gender of a passenger via passenger['Sex'].
    If the passenger is male, passenger['Sex'] will return a string "male".
    If the passenger is female, passenger['Sex'] will return a string "female".
    
    You can access the socioeconomic status of a passenger via passenger['Pclass']:
    High socioeconomic status -- passenger['Pclass'] is 1
    Medium socioeconomic status -- passenger['Pclass'] is 2
    Low socioeconomic status -- passenger['Pclass'] is 3

    You can access the age of a passenger via passenger['Age'].
    
    Write your prediction back into the "predictions" dictionary. The
    key of the dictionary should be the Passenger's id (which can be accessed
    via passenger["PassengerId"]) and the associated value should be 1 if the
    passenger survived or 0 otherwise. 

    For example, if a passenger is predicted to have survived:
    passenger_id = passenger['PassengerId']
    predictions[passenger_id] = 1

    And if a passenger is predicted to have perished in the disaster:
    passenger_id = passenger['PassengerId']
    predictions[passenger_id] = 0
    
    You can also look at the Titantic data that you will be working with
    at the link below:
    https://s3.amazonaws.com/content.udacity-data.com/courses/ud359/titanic_data.csv
    '''

    predictions = {}
    df = pandas.read_csv(file_path)
    for passenger_index, passenger in df.iterrows():
        passenger_id = passenger['PassengerId']
        # 
        # your code here
        # for example, assuming that passengers who are male
        # and older than 18 surived:
        #     if passenger['Sex'] == 'male' or passenger['Age'] < 18:
        #         predictions[passenger_id] = 1
        # 
        if (passenger['Sex'] == 'female') or (passenger['Pclass'] == 1 and passenger['Age'] < 18):
            predictions[passenger_id] = 1
        else:
            predictions[passenger_id] = 0
        
    return predictions


In [5]:
complex_heuristic(url)

{1: 0,
 2: 1,
 3: 1,
 4: 1,
 5: 0,
 6: 0,
 7: 0,
 8: 0,
 9: 1,
 10: 1,
 11: 1,
 12: 1,
 13: 0,
 14: 0,
 15: 1,
 16: 1,
 17: 0,
 18: 0,
 19: 1,
 20: 1,
 21: 0,
 22: 0,
 23: 1,
 24: 0,
 25: 1,
 26: 1,
 27: 0,
 28: 0,
 29: 1,
 30: 0,
 31: 0,
 32: 1,
 33: 1,
 34: 0,
 35: 0,
 36: 0,
 37: 0,
 38: 0,
 39: 1,
 40: 1,
 41: 1,
 42: 1,
 43: 0,
 44: 1,
 45: 1,
 46: 0,
 47: 0,
 48: 1,
 49: 0,
 50: 1,
 51: 0,
 52: 0,
 53: 1,
 54: 1,
 55: 0,
 56: 0,
 57: 1,
 58: 0,
 59: 1,
 60: 0,
 61: 0,
 62: 1,
 63: 0,
 64: 0,
 65: 0,
 66: 0,
 67: 1,
 68: 0,
 69: 1,
 70: 0,
 71: 0,
 72: 1,
 73: 0,
 74: 0,
 75: 0,
 76: 0,
 77: 0,
 78: 0,
 79: 0,
 80: 1,
 81: 0,
 82: 0,
 83: 1,
 84: 0,
 85: 1,
 86: 1,
 87: 0,
 88: 0,
 89: 1,
 90: 0,
 91: 0,
 92: 0,
 93: 0,
 94: 0,
 95: 0,
 96: 0,
 97: 0,
 98: 0,
 99: 1,
 100: 0,
 101: 1,
 102: 0,
 103: 0,
 104: 0,
 105: 0,
 106: 0,
 107: 1,
 108: 0,
 109: 0,
 110: 1,
 111: 0,
 112: 1,
 113: 0,
 114: 1,
 115: 1,
 116: 0,
 117: 0,
 118: 0,
 119: 0,
 120: 1,
 121: 0,
 122: 0,
 123: 0,
 

In [35]:
import numpy
import pandas
import statsmodels.api as sm

def custom_heuristic(file_path):
    '''
    You are given a list of Titantic passengers and their associated
    information. More information about the data can be seen at the link below:
    http://www.kaggle.com/c/titanic-gettingStarted/data

    For this exercise, you need to write a custom heuristic that will take
    in some combination of the passenger's attributes and predict if the passenger
    survived the Titanic diaster.

    Can your custom heuristic beat 80% accuracy?
    
    The available attributes are:
    Pclass          Passenger Class
                    (1 = 1st; 2 = 2nd; 3 = 3rd)
    Name            Name
    Sex             Sex
    Age             Age
    SibSp           Number of Siblings/Spouses Aboard
    Parch           Number of Parents/Children Aboard
    Ticket          Ticket Number
    Fare            Passenger Fare
    Cabin           Cabin
    Embarked        Port of Embarkation
                    (C = Cherbourg; Q = Queenstown; S = Southampton)
                    
    SPECIAL NOTES:
    Pclass is a proxy for socioeconomic status (SES)
    1st ~ Upper; 2nd ~ Middle; 3rd ~ Lower

    Age is in years; fractional if age less than one
    If the age is estimated, it is in the form xx.5

    With respect to the family relation variables (i.e. SibSp and Parch)
    some relations were ignored. The following are the definitions used
    for SibSp and Parch.

    Sibling:  brother, sister, stepbrother, or stepsister of passenger aboard Titanic
    Spouse:   husband or wife of passenger aboard Titanic (mistresses and fiancees ignored)
    Parent:   mother or father of passenger aboard Titanic
    Child:    son, daughter, stepson, or stepdaughter of passenger aboard Titanic
    
    Write your prediction back into the "predictions" dictionary. The
    key of the dictionary should be the passenger's id (which can be accessed
    via passenger["PassengerId"]) and the associating value should be 1 if the
    passenger survvied or 0 otherwise. 

    For example, if a passenger is predicted to have survived:
    passenger_id = passenger['PassengerId']
    predictions[passenger_id] = 1

    And if a passenger is predicted to have perished in the disaster:
    passenger_id = passenger['PassengerId']
    predictions[passenger_id] = 0
    
    You can also look at the Titantic data that you will be working with
    at the link below:
    https://s3.amazonaws.com/content.udacity-data.com/courses/ud359/titanic_data.csv
    '''

    predictions = {}
    df = pandas.read_csv(file_path)
    for passenger_index, passenger in df.iterrows():
        #
        # your code here
        #
        passenger_id = passenger['PassengerId']
        if passenger['Sex'] == 'female' and passenger['Pclass'] >= 2:
            predictions[passenger_id] = 1
        elif passenger['Sex'] == 'female' and (passenger['Parch'] > 3 or passenger['SibSp'] >= 3):
            predictions[passenger_id] = 0
        else:
            predictions[passenger_id] = 0
        
    return predictions


In [10]:
import seaborn as sns
%matplotlib inline

In [6]:
 df = pandas.read_csv(url)

In [7]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [25]:
df.groupby(['Pclass', 'Embarked', 'Sex'])['Survived'].agg(numpy.mean)

Pclass  Embarked  Sex   
1       C         female    0.976744
                  male      0.404762
        Q         female    1.000000
                  male      0.000000
        S         female    0.958333
                  male      0.354430
2       C         female    1.000000
                  male      0.200000
        Q         female    1.000000
                  male      0.000000
        S         female    0.910448
                  male      0.154639
3       C         female    0.652174
                  male      0.232558
        Q         female    0.727273
                  male      0.076923
        S         female    0.375000
                  male      0.128302
Name: Survived, dtype: float64

In [31]:
df.groupby(['Sex', 'Pclass'])['Survived'].agg(numpy.mean)

Sex     Pclass
female  1         0.968085
        2         0.921053
        3         0.500000
male    1         0.368852
        2         0.157407
        3         0.135447
Name: Survived, dtype: float64

In [30]:
df.groupby(['Sex', 'Embarked'])['Survived'].agg(numpy.mean)

Sex     Embarked
female  C           0.876712
        Q           0.750000
        S           0.689655
male    C           0.305263
        Q           0.073171
        S           0.174603
Name: Survived, dtype: float64

In [33]:
df.groupby(['Sex', 'Parch'])['Survived'].agg(numpy.mean)

Sex     Parch
female  0        0.788660
        1        0.766667
        2        0.612245
        3        0.750000
        4        0.000000
        5        0.250000
        6        0.000000
male    0        0.165289
        1        0.327586
        2        0.322581
        3        0.000000
        4        0.000000
        5        0.000000
Name: Survived, dtype: float64

In [34]:
df.groupby(['Sex', 'SibSp'])['Survived'].agg(numpy.mean)

Sex     SibSp
female  0        0.787356
        1        0.754717
        2        0.769231
        3        0.363636
        4        0.333333
        5        0.000000
        8        0.000000
male    0        0.168203
        1        0.310680
        2        0.200000
        3        0.000000
        4        0.083333
        5        0.000000
        8        0.000000
Name: Survived, dtype: float64

In [19]:
df.groupby(['Sex', 'Survived'])['Age'].agg(numpy.mean)

Sex     Survived
female  0           25.046875
        1           28.847716
male    0           31.618056
        1           27.276022
Name: Age, dtype: float64

In [20]:
df.groupby(['Sex', 'Survived'])['Fare'].agg(numpy.mean)

Sex     Survived
female  0           23.024385
        1           51.938573
male    0           21.960993
        1           40.821484
Name: Fare, dtype: float64

In [36]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [50]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [57]:
data = df.drop('Cabin', axis=1).dropna()[['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]

In [58]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 0 to 890
Data columns (total 8 columns):
Survived    712 non-null int64
Pclass      712 non-null int64
Sex         712 non-null object
Age         712 non-null float64
SibSp       712 non-null int64
Parch       712 non-null int64
Fare        712 non-null float64
Embarked    712 non-null object
dtypes: float64(2), int64(4), object(2)
memory usage: 50.1+ KB


In [59]:
y = data['Survived'].values
data.drop('Survived', axis=1, inplace=True)


In [64]:
dummy = pandas.get_dummies(data[['Sex', 'Embarked']])

data = pandas.concat([data, dummy], axis=1).drop(['Embarked', 'Sex'], axis=1)
X = data.values

In [65]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
X_train.shape, X_test.shape

((534, 10), (178, 10))

In [91]:
tree = DecisionTreeClassifier(max_depth=3).fit(X_train, y_train)

In [92]:
tree.score(X_train, y_train), tree.score(X_test, y_test)

(0.79962546816479396, 0.8258426966292135)

In [93]:
tree.feature_importances_

array([ 0.28692456,  0.08972602,  0.        ,  0.        ,  0.06154697,
        0.        ,  0.56180245,  0.        ,  0.        ,  0.        ])

In [94]:
from sklearn.tree import export_graphviz

In [95]:
export_graphviz(tree, 'tree.dot', feature_names=data.columns)

In [96]:
!dot -Tpng tree.dot -o tree.png

In [97]:
data.columns

Index(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Sex_female', 'Sex_male',
       'Embarked_C', 'Embarked_Q', 'Embarked_S'],
      dtype='object')

In [83]:
data.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,22.0,1,0,7.2500,0.0,1.0,0.0,0.0,1.0
1,1,38.0,1,0,71.2833,1.0,0.0,1.0,0.0,0.0
2,3,26.0,0,0,7.9250,1.0,0.0,0.0,0.0,1.0
3,1,35.0,1,0,53.1000,1.0,0.0,0.0,0.0,1.0
4,3,35.0,0,0,8.0500,0.0,1.0,0.0,0.0,1.0
